<a href="https://colab.research.google.com/github/atlas-github/Hansard_Analysis/blob/master/Hansard_WordCount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PyPDF2

     |████████████████████████████████| 81kB 2.2MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=62997482e03a7c4f0a10d54809f6e07f52891e03742baeb0dcfc10da5d1d5c7e
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [0]:
import PyPDF2
import pandas as pd

def word_count(filename):
  pdfFileObject = open(r'DR-05122019.pdf', 'rb')   ##need to modify this part to generic filename
  pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
  pageObject = pdfReader.getPage(0)
  #pdfFileObject.close()

  contents = list(pageObject.extractText().split(")\n \n \n"))
  pd.set_option('display.max_colwidth', None)
  df1 = pd.DataFrame(contents, columns = {"Agenda"})
  df1 = df1[:-1]
  #df1

  df2 = df1['Agenda'].str.split('Halaman', 1, expand=True)
  df2 = df2.rename(columns = {0: "Agenda", 1: "Page"})
  df2['Agenda'] = df2['Agenda'].str.replace('\n', '')
  df2['Agenda'] = df2['Agenda'].str[:-2]

  agg_word_count = []

  word_count = []

  for i in range(len(df2)-1): 
    for j in range(int(df2.iloc[i, 1]), int(df2.iloc[i+1, 1])):
      pageObject2 = pdfReader.getPage(j)
      word_count_page = len(pageObject2.extractText().split())
      word_count.append(word_count_page)
    agg_word_count.append(sum(word_count))

  topic_word_count = []

  for i in range(len(agg_word_count)):
    topic_word_count.append(max(agg_word_count[i] - agg_word_count[i-1], 0))

  df_word_count = pd.DataFrame(topic_word_count, columns = {'Word_Count'})
  initial = pd.DataFrame([0], columns = {"Word_Count"})
  df_word_count_final = pd.concat([initial, df_word_count], ignore_index = True)
  df_word_count_merged = pd.concat([df2, df_word_count_final], axis = 1)
  return(df_word_count_merged)

In [108]:
word_count('DR-04122019.pdf')

,Agenda,Page,Word_Count
0,"Naskhah belum disemak PENYATA RASMI PARLIMEN DEWAN RAKYAT PARLIMEN KEEMPAT BELAS PENGGAL KEDUA MESYUARAT KETIGA Bil. 68 Khamis 5 Disember 2019 K A N D U N G A N PEMASYHURAN TUAN YANG DI-PETUA: - Hari Terakhir Setiausaha Dewan Rakyat, Datuk Roosme binti Hamzah Mengendalikan Urusan Dewan Rakyat",1,0
1,USUL MENANGGUHKAN BACAAN KALI YANG KEDUA DAN KETIGA RANG UNDANG-UNDANG DI BAWAH P.M. 62,1,0
2,JAWAPAN-JAWAPAN MENTERI BAGI PERTANYAAN-PERTANYAAN,2,224
3,JAWAPAN-JAWAPAN LISAN BAGI PERTANYAAN-PERTANYAAN,8,2501
4,USUL: Waktu Mesyuarat dan Urusan Dibebaskan Daripada Peraturan Mesyuarat,37,12269
5,PETUA TUAN YANG DI-PERTUA: -,37,0
6,USUL MENTERI DI JABATAN PERDANA MENTERI DI BAWAH P.M 27(3): - Laporan Tahunan dan Penyata Kewangan Suruhanjaya Hak Asasi Manusia Malaysia (SUHAKAM) 2018,42,1878
7,USUL-USUL MENTERI KEWANGAN: - Akta Kastam 1967,100,24052
8,- Akta Eksais 1976,104,1669
9,- Akta Cukai Perkhidmatan 2018,105,378


In [107]:
import timeit
starttime = timeit.default_timer()
word_count('DR-04122019.pdf')
timeit.default_timer() - starttime

4.750962561000051